# TARPES analysis

## note
As the real tarpes data is really larlge (making it impractical to include this package), so here we create a mock Xarray resembling the data and use it for explanation.

The mockl data is a list that contains xarray:
* 1000 xarray objects.  Each xarray represents the single set at a certain position of the delay line.
* 20 x 20 matrix data
* Angle (-10deg to 10deg) and energy (5 eV to 6eV)
* "position" attributes, for the delay line positilon (100.0mm to 103.00 mm), which is converted to delay time.

In [ ]:
import numpy as np
import xarray as xr
from scipy.special import erf
import matplotlib.pyplot as plt

import arpes

In [ ]:
def temporal_from_rate(t: float, g: float, sigma: float, k_ex: float, t0: float=0) -> float:
    """Temporal profile from a rate equation, which is used in (for example)
    10.1021/acs.nanolett.3c03251
    """
    return (g/2) * np.exp(k_ex * t0 + sigma**2 * (k_ex**2)/2) * np.exp(-k_ex * t) * (erf((t-t0+(sigma**2)*k_ex)/(sigma * np.sqrt(2)))+ erf((t0+(sigma**2)*k_ex)/(sigma*np.sqrt(2))))

In [ ]:
n_data = 150
position=np.linspace(100, 103, n_data)
delaytime = np.linspace(-100E-15, 2500E-15, n_data)
rng = np.random.default_rng(42)
noise = rng.normal(loc=0, scale=.01, size=n_data)
tempo_intensity = temporal_from_rate(t = delaytime,
                        g = 1,
                        sigma = 50E-15,
                        k_ex = 2E12,
                        t0=0.2E-12) + noise + 0.02
# plt.scatter(delaytime, tempo_intensity)

In [ ]:
mock_tarpes = [xr.DataArray(data = np.arange(400).reshape(20,20) * tempo_intensity[i]  ,
                            dims=["phi", "eV"],
                            coords ={"phi": np.linspace(np.deg2rad(-10), np.deg2rad(10), 20), "eV":np.linspace(5, 6, 20)}, 
                            attrs = {"position": position[i], "id":int(i+1) }) for i in range(n_data)]

## tarpes analysis 

### find_t_for_max_intensity

In [ ]:
tarpes_ = arpes.analysis.tarpes.build_crosscorrelation(mock_tarpes, delayline_dim="position", delayline_origin=100.31)

In [ ]:
arpes.analysis.tarpes.find_t_for_max_intensity(tarpes_)

In [ ]:
sum_dims = set(tarpes_.dims)
sum_dims.remove("delay")
sum_dims.remove("eV")

In [ ]:
summed = tarpes_.sum(list(sum_dims)).sel(eV=slice(None, None)).mean("eV")

In [ ]:
summed.plot()

### relative_change

In [ ]:
arpes.analysis.tarpes.relative_change(tarpes_)